<a href="https://colab.research.google.com/github/milvus-io/bootcamp/blob/master/integration/build_RAG_with_milvus_and_contextual_ai_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>   <a href="https://github.com/milvus-io/bootcamp/blob/master/integration/build_RAG_with_milvus_and_contextual_ai_parser.ipynb" target="_blank">
    <img src="https://img.shields.io/badge/View%20on%20GitHub-555555?style=flat&logo=github&logoColor=white" alt="GitHub Repository"/>


# Build RAG with Milvus and Contextual AI

[Contextual AI Parser](https://docs.contextual.ai/api-reference/parse/parse-file?utm_campaign=Parse-api-integration&utm_source=milvus&utm_medium=github&utm_content=notebook) is a cloud-based document parsing service that excels at extracting structured information from PDFs, DOC/DOCX, and PPT/PPTX files. It provides high-quality markdown extraction with document hierarchy preservation and advanced table extraction, making it ideal for RAG applications.

In this tutorial, we'll show you how to build a Retrieval-Augmented Generation (RAG) pipeline using Milvus and Contextual AI Parser. The pipeline integrates Contextual AI Parser for document parsing, Milvus for vector storage, and OpenAI for generating insightful, context-aware responses.


## Preparation

### Dependencies and Environment

To start, install the required dependencies by running the following command:


In [ ]:
! pip install --upgrade "pymilvus[milvus_lite]" contextual-client openai requests rich


> If you are using Google Colab, to enable dependencies just installed, you may need to **restart the runtime** (click on the "Runtime" menu at the top of the screen, and select "Restart session" from the dropdown menu).


### Setting Up API Keys

We will use Contextual AI for document parsing and OpenAI as the LLM in this example. You should prepare the [CONTEXTUAL_API_KEY](https://docs.contextual.ai/user-guides/beginner-guide?utm_campaign=Parse-api-integration&utm_source=milvus&utm_medium=github&utm_content=notebook) and [OPENAI_API_KEY](https://platform.openai.com/docs/quickstart) as environment variables.

If you're running this notebook in Google Colab, you can add your API keys as secrets. The code below dynamically handles both Colab secrets and environment variables.


In [ ]:
import os

# API key variable names
contextual_api_key_var = "CONTEXTUAL_API_KEY"
openai_api_key_var = "OPENAI_API_KEY"

# Fetch API keys
try:
    # If running in Colab, fetch API keys from Secrets
    import google.colab
    from google.colab import userdata
    contextual_api_key = userdata.get(contextual_api_key_var)
    openai_api_key = userdata.get(openai_api_key_var)
    
    if not contextual_api_key:
        raise ValueError(f"Secret '{contextual_api_key_var}' not found in Colab secrets.")
    if not openai_api_key:
        raise ValueError(f"Secret '{openai_api_key_var}' not found in Colab secrets.")
except ImportError:
    # If not running in Colab, fetch API keys from environment variables
    contextual_api_key = os.getenv(contextual_api_key_var)
    openai_api_key = os.getenv(openai_api_key_var)
    
    if not contextual_api_key:
        raise EnvironmentError(
            f"Environment variable '{contextual_api_key_var}' is not set. "
            "Please define it before running this script."
        )
    if not openai_api_key:
        raise EnvironmentError(
            f"Environment variable '{openai_api_key_var}' is not set. "
            "Please define it before running this script."
        )

os.environ["CONTEXTUAL_API_KEY"] = contextual_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key


### Prepare the LLM and Embedding Model

We initialize the OpenAI client for embeddings and Contextual AI client for GLM.


Define a function to generate text embeddings using OpenAI client. We use the [text-embedding-3-small](https://platform.openai.com/docs/guides/embeddings) model as an example.


In [ ]:
from openai import OpenAI
from contextual import ContextualAI

openai_client = OpenAI()
contextual_client = ContextualAI(api_key=contextual_api_key)


In [ ]:
def emb_text(text):
    return (
        openai_client.embeddings.create(input=text, model="text-embedding-3-small")
        .data[0]
        .embedding
    )


Generate a test embedding and print its dimension and first few elements.


In [ ]:
test_embedding = emb_text("This is a test")
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])


## Process Data Using Contextual AI Parser

Contextual AI Parser can parse various document formats into structured markdown with document hierarchy preservation. The parser handles complex documents with images, tables, and hierarchical structures, providing multiple output formats including:
- `markdown-document`: Single concatenated markdown output
- `markdown-per-page`: Page-by-page markdown output
- `blocks-per-page`: Structured JSON with document hierarchy

For a full list of supported input and output formats, please refer to [the official documentation](https://docs.contextual.ai/api-reference/parse/parse-file?utm_campaign=Parse-api-integration&utm_source=milvus&utm_medium=github&utm_content=notebook).

In this tutorial, we will parse two distinct document types: a research paper and a table-rich document. We'll use the `blocks-per-page` format to extract structured chunks suitable for downstream RAG tasks.


In [ ]:
import requests
from time import sleep

# Documents to parse with Contextual AI
documents = [
    {
        "url": "https://arxiv.org/pdf/1706.03762",
        "title": "Attention Is All You Need",
        "type": "research_paper",
        "description": "Seminal transformer architecture paper that introduced self-attention mechanisms"
    },
    {
        "url": "https://raw.githubusercontent.com/ContextualAI/examples/refs/heads/main/03-standalone-api/04-parse/data/omnidocbench-text.pdf",
        "title": "OmniDocBench Dataset Documentation", 
        "type": "table_rich_document",
        "description": "Dataset documentation with large tables demonstrating table extraction capabilities"
    }
]

# Parse documents and extract text chunks
texts = []
for doc in documents:
    print(f"Parsing: {doc['title']}")
    
    # Download file from URL
    file_content = requests.get(doc["url"]).content
    
    # Submit parse job
    with open("temp_file.pdf", "wb") as f:
        f.write(file_content)
    
    with open("temp_file.pdf", "rb") as fp:
        response = contextual_client.parse.create(
            raw_file=fp,
            parse_mode="standard",
            enable_document_hierarchy=True,
            enable_split_tables=False,
            figure_caption_mode="concise"
        )
    
    job_id = response.job_id
    
    # Wait for job to complete
    while True:
        status = contextual_client.parse.job_status(job_id)
        if status.status == "completed":
            break
        elif status.status == "failed":
            raise Exception("Parse job failed")
        sleep(30)  # Wait 30 seconds before checking again
    
    # Get results
    results = contextual_client.parse.job_results(job_id, output_types=["blocks-per-page"])
    
    # Extract text from blocks
    for page in results.pages:
        for block in page.blocks:
            if block.markdown:
                texts.append(block.markdown)
    
    print(f"  Extracted {len([b for page in results.pages for b in page.blocks if b.markdown])} text blocks")

print(f"\nTotal chunks extracted: {len(texts)}")

## Load Data into Milvus

### Create the collection


In [ ]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")
collection_name = "my_rag_collection"


> As for the argument of `MilvusClient`:
> - Setting the `uri` as a local file, e.g.`./milvus.db`, is the most convenient method, as it automatically utilizes [Milvus Lite](https://milvus.io/docs/milvus_lite.md) to store all data in this file.
> - If you have large scale of data, you can set up a more performant Milvus server on [docker or kubernetes](https://milvus.io/docs/quickstart.md). In this setup, please use the server uri, e.g.`http://localhost:19530`, as your `uri`.
> - If you want to use [Zilliz Cloud](https://zilliz.com/cloud), the fully managed cloud service for Milvus, adjust the `uri` and `token`, which correspond to the [Public Endpoint and Api key](https://docs.zilliz.com/docs/on-zilliz-cloud-console#free-cluster-details) in Zilliz Cloud.


Check if the collection already exists and drop it if it does.


In [ ]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)


Create a new collection with specified parameters.

If we don't specify any field information, Milvus will automatically create a default `id` field for primary key, and a `vector` field to store the vector data. A reserved JSON field is used to store non-schema-defined fields and their values.


In [ ]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # Inner product distance
    # Strong consistency waits for all loads to complete, adding latency with large datasets
    # consistency_level="Strong",  # Supported values are (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`). See https://milvus.io/docs/consistency.md#Consistency-Level for more details.
)


### Insert data

In [ ]:
from tqdm import tqdm

data = []

for i, chunk in enumerate(tqdm(texts, desc="Processing chunks")):
    embedding = emb_text(chunk)
    data.append({"id": i, "vector": embedding, "text": chunk})

milvus_client.insert(collection_name=collection_name, data=data)


## Build RAG

### Retrieve data for a query

Let's specify a query question about the parsed documents.


In [ ]:
question = "What is the transformer architecture and how does self-attention work?"


Search for the question in the collection and retrieve the semantic top-3 matches.


In [ ]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=[emb_text(question)],
    limit=3,
    search_params={"metric_type": "IP", "params": {}},
    output_fields=["text"],
)


Let's take a look at the search results of the query


In [ ]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))


### Use LLM to get a RAG response

Convert the retrieved documents into a string format.


In [ ]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)


Define system and user prompts for the Language Model. This prompt is assembled with the retrieved documents from Milvus.


In [ ]:
SYSTEM_PROMPT = """
Human: You are an AI assistant. You are able to find answers to the questions from the contextual passage snippets provided.
"""
USER_PROMPT = f"""
Use the following pieces of information enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
<context>
{context}
</context>
<question>
{question}
</question>
"""


Use OpenAI ChatGPT to generate a response based on the prompts.


In [ ]:
response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)
